In [2]:
## import necessary Libaries for this project
import tensorflow as tf
from tensorflow import keras
import numpy as np
from kerastuner.tuners import RandomSearch

In [3]:
## to initialize the datasets from fashion-mnist
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
fashion_mnist

<module 'tensorflow.keras.datasets.fashion_mnist' from 'C:\\Users\\HP\\Anaconda3\\envs\\envtensorflow\\lib\\site-packages\\tensorflow\\keras\\datasets\\fashion_mnist\\__init__.py'>

In [5]:
## to split the datasets into train and test images
(train_images,train_labels),(test_images,test_labels)  = fashion_mnist.load_data()

In [6]:
## to normalize the pixel 0 to 1 values.
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
## to resize the images
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
    
    model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
    return model


In [10]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output\Mnist Fashion\oracle.json


In [11]:
tuner_search.search_space_summary()

Search space summary
Default search space size: 6
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
conv_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': None}
conv_2_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [12]:
## best hyperparameter
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [02h 21m 25s]
val_accuracy: 0.8690000176429749

Best val_accuracy So Far: 0.9208333492279053
Total elapsed time: 06h 11m 38s
INFO:tensorflow:Oracle triggered exit


In [17]:
## to get the best model usimg hyparameter
model = tuner_search.get_best_models(num_models=1)[0]
model

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        41520     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3096688   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,140,298
Trainable params: 3,140,298
Non-trainable params: 0
_________________________________________________________________


In [20]:
## retrained the images consedering the best model from above
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 367s 217ms/step - loss: 0.1058 - accuracy: 0.9608 - val_loss: 0.2958 - val_accuracy: 0.9122
Epoch 5/10
1688/1688 [==============================] - 313s 185ms/step - loss: 0.0695 - accuracy: 0.9745 - val_loss: 0.3358 - val_accuracy: 0.9105
Epoch 6/10
1688/1688 [==============================] - 282s 167ms/step - loss: 0.0451 - accuracy: 0.9839 - val_loss: 0.3741 - val_accuracy: 0.9122
Epoch 7/10
1688/1688 [==============================] - 284s 168ms/step - loss: 0.0344 - accuracy: 0.9882 - val_loss: 0.4302 - val_accuracy: 0.9117
Epoch 8/10
1688/1688 [==============================] - 294s 174ms/step - loss: 0.0264 - accuracy: 0.9904 - val_loss: 0.4437 - val_accuracy: 0.9160
Epoch 9/10
1688/1688 [==============================] - 377s 223ms/step - loss: 0.0225 - accuracy: 0.9922 - val_loss: 0.5153 - val_accuracy: 0.9142
Epoch 10/10
1688/1688 [==============================] - 434s 257ms/step - loss: 0.0203 - accuracy: 0.9930 - val